In [1]:
import torch
import argparse
import time
import math
import os, sys
import itertools
import pickle
from tqdm import tqdm
import numpy as np
import os.path as osp

In [2]:
import torch

# 创建形状为 (n,) 的标签张量 label
label = torch.tensor([0, 1, 1, 0, 1])

# 找到所有标签值为 1 的元素的索引
index = torch.nonzero(label == 1).squeeze()

# 输出索引
print(index)


tensor([1, 2, 4])


In [3]:
torch.cuda.device_count()

1

In [4]:
def loadpkl(path):
    with open(path, 'rb') as f:
        rst=pickle.load(f)
    return rst

In [5]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/full_id_abs_title_cite_ref_author.pkl"
full_data=loadpkl(path)

In [6]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/train.pkl"
train=loadpkl(path)

In [8]:
print(len(train))

172610


In [10]:
cnt=0
train_cite_pair=[]
for paper in train:
    if len(full_data[paper]['abstract'])>0:
        for ref in full_data[paper]['references']:
            if ref in full_data.keys():
                train_cite_pair.append((paper,ref))
print(len(train_cite_pair))

5288766


In [11]:
train_cite_pair[0]

('7502372bac32d9967eeb4bf68a430a74dadf748c',
 'fe2b6e2abc22bdd4338953ec47eea276c75c9819')

In [6]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/train_cite_pair.pkl"
train_cite_pair=loadpkl(path)

In [14]:
print(len(train_cite_pair))

5288766


In [12]:
train_cite_pair[0]

('9dad5f3b491fba9496c90bb7cfe10d1c0f00fadc',
 '402f850dff86fb601d34b2841e6083ac0f928edd')

In [10]:
train_cite_pair=list(set(train_cite_pair))

In [13]:
new_train=[]
for k in train_cite_pair:
    query=k[0]
    key=k[1]
    if query in full_data.keys() and key in full_data.keys():
        if full_data[query]['abstract'] is not None and full_data[key]['abstract'] is not None:
            new_train.append((query,key))
    

In [14]:
print(len(new_train))

4201091


In [15]:
new_train[0]

('9dad5f3b491fba9496c90bb7cfe10d1c0f00fadc',
 '402f850dff86fb601d34b2841e6083ac0f928edd')

In [13]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/train_cite_pair.pkl"
with open(path,'wb') as f:
    pickle.dump(train_cite_pair,f)

In [17]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/dev_cite_pair.pkl"
dev_cite_pair=loadpkl(path)

In [18]:
dev_cite_pair=list(set(dev_cite_pair))

In [19]:
print(len(dev_cite_pair))

35540


In [20]:
new_dev=[]
for k in dev_cite_pair:
    query=k[0]
    key=k[1]
    if query in full_data.keys() and key in full_data.keys():
        if full_data[query]['abstract'] is not None and full_data[key]['abstract'] is not None:
            new_dev.append((query,key))

In [21]:
print(len(new_dev))

29218


In [22]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/dev_cite_pair.pkl"
with open(path,'wb') as f:
    pickle.dump(new_dev,f)

In [23]:
cnt=0
train_key=set()
for k in new_train:
    train_key.add(k[0])
for k in new_dev:
    if k[0] in train_key:
        cnt+=1
print(cnt)

0


In [24]:
print(len(train_key))

149259


In [26]:
list(train_key)[0]

'07f8c9d9b8c438dc200a03ae44217aa4af2e25a2'

In [27]:
new_dev[0][0]

'b5e23ed419347d125874d721d82151e2e32e70bc'

In [ ]:
test_gold=loadpkl("/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_test_gold.pkl")

In [29]:
cnt=0
total=0
for k,v in test_gold.items():
    for pid in v:
        if pid in full_data.keys() and full_data[pid]['abstract'] is not None:
            cnt+=1
        total+=1

In [30]:
print(cnt)
print(total)

31465
38440


In [31]:
save_dir='/share/data/mei-work/kangrui/github/ssref/result/sentence-transformers_all-MiniLM-L6-v2/2023-04-03T04-11-59'
key_embed_path=osp.join(osp.join(save_dir,'test_result'),'eval_key_id2embed.pkl')
test_embed_path=osp.join(osp.join(save_dir,'test_result'),'eval_test_id2embed.pkl')

In [32]:
key_embed_path

'/share/data/mei-work/kangrui/github/ssref/result/sentence-transformers_all-MiniLM-L6-v2/2023-04-03T04-11-59/test_result/eval_key_id2embed.pkl'

In [33]:
osp.exists(key_embed_path)

True

In [34]:
osp.exists(test_embed_path)

True